Metabolic EP
=====

Load julia module
-----

In [43]:
# eventually, install repository with import Pkg; Pkg.clone("https://github.com/anna-pa-m/Metabolic-EP/","MetabolicEP.jl")
#import Pkg; Pkg.clone("https://github.com/anna-pa-m/Metabolic-EP/","MetabolicEP.jl")
#using MetabolicEP
include("src/MetabolicEP.jl")

Main.MetabolicEP

Load test metabolic model
------

In [19]:
#met=MetabolicEP.ReadMatrix(Pkg.Dir.path() * "/MetabolicEP/test/ecoli_core_model.mat");
met=MetabolicEP.ReadMatrix("test/ecoli_core_model.mat");

Run EP
-----

In [20]:
r = MetabolicEP.metabolicEP(met.S,met.b,met.lb,met.ub,beta=1e9);
display(r.μ)

Analyzing a 72 x 95 stoichiometric matrix.
Converging with β=1.0e9 maxth=1.0e-6 maxiter=2000:
it = 341 ɛav = 9.66e-07 ɛvar = 4.08e-08 ɛμ = 1.79e-06 ɛs = 9.40e-08                  ɛμ = 2.65e-02 ɛs = 1.75e-03                 e-06 ɛs = 4.66e-07                 


95-element Array{Float64,1}:
  -1.9939969679570093 
  -1.0552173803738025 
  -1.4410949781534763 
   9.31697692715414   
   9.318877875880977  
  -1.441768583322908  
  10.644874131167954  
   0.19692096539851645
  -0.5517449556395375 
  -0.9396736351032438 
 -72.18550809309693   
  49.37506591228082   
  -0.5878520734675554 
   ⋮                  
  -1.0468632483001472 
   4.630343750206908  
  -2.35421590802529   
   5.3157879549604345 
   8.048668531915112  
 507.51748121312335   
  -3.5794285789622617 
   2.323976466704064  
 -68.27312112928051   
   2.32100067197318   
   2.3115057743060548 
   7.279246662043479  

In [49]:
# Beta = Inf 

#import Pkg; Pkg.clone("https://github.com/blegat/RowEchelon.jl.git", "RowEchelon.jl")
#using RowEchelon.jl
#include("/home/muntoni/src/RowEchelon.jl/src/RowEchelon.jl")
#Snew = RowEchelon.rref!(Matrix(met.S))
using DelimitedFiles

# INPUT ? S = [I | M] or S = M ? 
idxdep, idxnew,idxmap,Snew,bnew = MetabolicEP.echelonize(Matrix(met.S),met.b)
r0 = MetabolicEP.metabolicEP(Snew,bnew,met.lb,met.ub,beta=Inf);
display(r0.μ)

Analyzing a 67 x 95 stoichiometric matrix.
Converging with β=Inf maxth=1.0e-6 maxiter=2000:


scra1 = -3.162772141850787e29 scra2 = Inf


it = 2000 ɛav = 1.41e+73 ɛvar = 0.00e+00 ɛμ = 7.76e+88 ɛs = 1.60e+41                  ɛs = 4.45e+46                 88 ɛs = 1.97e+41                 


95-element Array{Float64,1}:
  3.171203571905595e89 
 -4.083086964343085e88 
 -1.2796826131762322e89
 -8.633684655850779e87 
 -8.633684655850779e87 
 -1.2796826131762322e89
 -2.1790656190096558e89
  7.664972951431435e90 
  6.631982040303006e89 
 -5.329358638965989e87 
 -1.6232447158001796e91
 -8.633684655850779e87 
  2.40696459026451e89  
  ⋮                    
 -2.301071041130132e91 
 -1.2127102333437572e92
 -2.824473897375961e88 
 -4.2687190411831924e91
  9.441764910707217e91 
 -6.3245407280655e91   
  2.3223597067171083e92
 -1.6905443360911605e92
  1.4246985603378833e91
  6.249694447389881e91 
  3.07962670083576e91  
  2.1877758852341304e91

In [23]:
using Plots
Plots.plotly()
scatter(r.μ, r0.μ)

ArgumentError: ArgumentError: At least one finite value must be provided to formatter.

Plot marginals
-----
    
Marginals are truncated Normal



$$P(\nu_i)\propto \mathbb{I}[l_i\leq\nu_i\leq u_i]e^{-\frac{(\nu_i-\mu_i)^2}{2s_i}}$$

In [10]:
#if you miss these packages, use Pkg.add("Distributions); Pkg.add("Gadfly")
using Distributions, Gadfly

function plotmarg(i)
    μ,s,l,u=r.μ[i],r.σ[i],met.lb[i],met.ub[i]
    d=Normal(μ,√s);
    D=Truncated(d,l,u);
    #choose bounds to have a sensible plotting range
    m=max(pdf(d,l),pdf(d,u),(l<μ<u) ? 1/sqrt(2pi*s) : 0);
    v=sqrt(-2*s*log(sqrt(2pi*s)*1e-5m));
    l1,l2=max(l,μ-v),min(u,μ+v);
    return Gadfly.plot(x->pdf(D,x),l1,l2,Guide.YLabel(""),Guide.XLabel(met.rxns[i]))
end

#build plot grid with R rows and 5 columns (up to R=19)
R=19
P=[plotmarg(i) for i=1:5R]
Gadfly.set_default_plot_size(30cm,R*5cm)
Gadfly.gridstack(reshape(P,R,5))

┌ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]
└ @ Base loading.jl:1186
Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /home/muntoni/src/julia/src/locks.h:138 [inlined]
jl_typeinf_end at /home/muntoni/src/julia/src/gf.c:2594
typeinf_ext at ./compiler/typeinfer.jl:584
typeinf_ext at ./compiler/typeinfer.jl:620
jfptr_typeinf_ext_1 at /home/muntoni/src/julia/usr/lib/julia/sys.so (unknown line)
jl_apply_generic at /home/muntoni/src/julia/src/gf.c:2209
jl_apply at /home/muntoni/src/julia/src/julia.h:1571 [inlined]
jl_type_infer at /home/muntoni/src/julia/src/gf.c:277
jl_compile_method_internal at /home/muntoni/src/julia/src/gf.c:1809 [inlined]
jl_fptr_trampoline at /home/muntoni/src/julia/src/gf.c:1853
jl_apply_generic at /home/muntoni/src/julia/src/gf.c:2209
do_call at /home/muntoni/src/julia/src/interpreter.c:323
eval_value at /home/muntoni/src/julia/src/interpreter.c:411
eval_stmt_value at /home/muntoni/src/julia/src/i

MethodError: MethodError: no method matching isless(::Float64, ::Nothing)
Closest candidates are:
  isless(::Float64, !Matched::Float64) at float.jl:459
  isless(!Matched::Missing, ::Any) at missing.jl:70
  isless(::AbstractFloat, !Matched::AbstractFloat) at operators.jl:148
  ...